# MLOPS Camp - Homework #1
https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#homework

## Imports

In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

## Read data

In [2]:
january_data = pd.read_parquet("../../data/tlc_data/yellow_tripdata_2023-01.parquet")
february_data = pd.read_parquet("../../data/tlc_data/yellow_tripdata_2023-02.parquet")

In [3]:
january_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
february_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


## Q1. Downloading the data
Read the data for January. How many columns are there?

In [5]:
len(january_data.columns)

19

## Q2. Computing duration
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [6]:
january_data["duration"] = (january_data["tpep_dropoff_datetime"] - january_data["tpep_pickup_datetime"]).dt.total_seconds()/60.0

In [7]:
january_data["duration"].std()

42.59435124195458

## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [8]:
with_outliers = len(january_data)
with_outliers

3066766

In [9]:
january_data_without_outliers = january_data.loc[january_data.loc[:, "duration"].between(1, 60), :]
without_outliers = len(january_data_without_outliers)
without_outliers

3009173

In [10]:
without_outliers / with_outliers * 100

98.1220282212598

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [11]:
one_hot_columns = january_data_without_outliers.loc[:, ["PULocationID", "DOLocationID"]].astype("string")
one_hot_columns.head()

,PULocationID,DOLocationID
0,161,141
1,43,237
2,48,238
3,138,7
4,107,79


In [12]:
one_hot_list = one_hot_columns.to_dict('records')

In [13]:
encoder = DictVectorizer().fit(one_hot_list)

In [14]:
january_encoded_data = encoder.transform(one_hot_list)
january_encoded_data.shape[1]

515

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [15]:
model = LinearRegression()
model.fit(january_encoded_data, january_data_without_outliers["duration"])

LinearRegression()

In [16]:
jan_duration_predicted = model.predict(january_encoded_data)

In [17]:
root_mean_squared_error(january_data_without_outliers["duration"], jan_duration_predicted)

7.649261937621321

## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [18]:
february_data["duration"] = (february_data["tpep_dropoff_datetime"] - february_data["tpep_pickup_datetime"]).dt.total_seconds()/60.0
february_data = february_data.loc[february_data.loc[:, "duration"].between(1, 60), :]

In [19]:
feb_one_hot_columns = february_data.loc[:, ["PULocationID", "DOLocationID"]].astype("string")
feb_one_hot_list = feb_one_hot_columns.to_dict('records')

In [20]:
feb_encoded_data = encoder.transform(feb_one_hot_list)

In [21]:
feb_duration_predicted = model.predict(feb_encoded_data)

In [22]:
root_mean_squared_error(february_data["duration"], feb_duration_predicted)

7.811817646307258